## Extract the sorted peaks from FOD

In [3]:
import numpy as np
from utils.optimize_util import *
import nibabel as nib
from utils.extract_util import sorted_peaks
from utils.data_io import read_mni_obj

FOD_data = nib.load('example/FOD.nii.gz').get_fdata()[..., 0:45]
ctx_mask_fname = 'example/lh_ctx_mask.nii.gz'
ctx_mask = nib.load(ctx_mask_fname).get_fdata()
exp_params = parse_params('configs/__rad_est_default_params_hcp.yaml')
sphere_fpath = 'sphere5120.obj'

_,sphere_V,_, sphere_F = read_mni_obj(sphere_fpath)

peaks = sorted_peaks(FOD_data, sphere_V, sphere_F, exp_params['max_peaks'], ctx_mask, exp_params['relative_threshold'], exp_params['min_separation_angle'], peak_threshold=exp_params['peak_threshold'])
peaks_flatted = peaks.reshape(*peaks.shape[0:3], -1)

peaks_fname = 'example/lh_peaks.nii.gz'
save_nifty_data(peaks_fname, peaks_flatted, ref_file=ctx_mask_fname)

## Load Precomputed neighbor_map

In [4]:
from utils.stencil_util import get_stencil
neighbor_map = nib.load('example/lh_neighbor_6.nii.gz').get_fdata()
n_neighbor = exp_params['n_neighbor']
stencil_order = get_stencil(n_neighbor)


## Optimize 

In [6]:
normal = nib.load('example/lh_ctx_normal.nii.gz').get_fdata()

# peaks_fname = 'example/lh_peaks.nii.gz'
# peaks = nib.load(peaks_fname).get_fdata().reshape(ctx_mask.shape[0:3]+(exp_params['max_peaks'],3))

exp_params = parse_params('configs/__rad_est_default_params_hcp.yaml')

results = optimize_one(peaks, normal, exp_params, neighbor_mask=neighbor_map, stencil_order=stencil_order)
out_prefix = 'example/PeakExtr/lh'
save_results_nifty(results, ref_fname=ctx_mask_fname, out_prefix=out_prefix)


Epoch 1, Loss: -0.5655477658349415
Epoch 2, Loss: -0.6573614516055399
